# Is saving files ACID?

In [ ]:
from laminci.db import setup_local_test_postgres

pgurl = setup_local_test_postgres()
!lamin init --storage "./test-acid" --db={pgurl}

In [ ]:
import lamindb as ln
from upath import UPath
import pytest

ln.settings.verbosity = 3  # show hints

## Save error due to failed upload to storage

Let's try to ingest a data object to a storage location without permission.

In [ ]:
# Create data object
adata = ln.dev.datasets.anndata_mouse_sc_lymph_node()
file = ln.File(adata, name="Mouse Lymph Node scRNA-seq")

In [ ]:
# hack storage path with corrupt path
ln.setup.settings.storage._root = UPath("s3://nf-core-awsmegatests")

In [ ]:
ln.settings.storage

In [ ]:
# try to save
with pytest.raises(RuntimeError) as error:
    ln.save(file)
print(error.exconly())

In [ ]:
# try to save with alternative syntax
with pytest.raises(RuntimeError) as error:
    file.save()
print(error.exconly())

Let's check that no metadata records were added to the database.

In [ ]:
files = ln.File.select().all()
assert len(files) == 0

## Save error due to failed database transaction

Let's try to add the same `Project` record twice, violating the primary key unique constraint.

In [ ]:
from django.db.utils import IntegrityError

project = ln.Project(name="test-project")
project.save()
conflict_project = ln.Project(id=project.id, name="conflict-project")
with pytest.raises(IntegrityError) as error:
    conflict_project.save()
print(error.exconly())

## Save failure during list-based ingestion

### Error during metadata save

In [ ]:
filepath = ln.dev.datasets.file_jpg_paradisi05()
file = ln.File(filepath)
files = [file, "this is not a BaseORM"]
with pytest.raises(Exception) as error:
    ln.save(files)
print(error.exconly())

Nothing got saved:

In [ ]:
files = ln.select(ln.File).all()
assert len(files) == 0

## Error in one of the uploads

If a list of data objects is passed to `ln.save()` and the upload of one of these data objects fails, the successful uploads are maintained and a `RuntimeError` is raised, listing the successfully uploaded data objects up until that point.

Need a proper test here.

## Change between working storage locations

The following is here in this notebook, because we have a postgres instance to test multi-storage. 

In [ ]:
ln.settings.storage = "./test-acid"

In [ ]:
ln.settings.storage = "s3://lamindb-ci"

In [ ]:
assert ln.setup.settings.storage.is_cloud
assert ln.setup.settings.storage.root_as_str == "s3://lamindb-ci"
# root.fs contains the underlying fsspec filesystem
assert (
    ln.setup.settings.storage.root.fs.cache_regions  # set by lamindb to True for s3 by default
)

In [ ]:
ln.settings.storage = "s3://lamindb-ci", dict(cache_regions=False)

In [ ]:
# test cache_regions
assert not ln.setup.settings.storage.root.fs.cache_regions

In [ ]:
assert len(ln.Storage.select().list()) == 2

In [ ]:
!docker stop pgtest && docker rm pgtest
!lamin delete pgtest
!rm -r ./test-acid